# Projeto SQL - Tripleten sprint 14  

O coronavírus pegou o mundo todo de surpresa mudando a rotina das pessoas. Os moradores das cidades já não passavam mais seu tempo livre fora de casa, indo a cafés, e shoppings; a maioria ficou em casa lendo livros. Isso chamou a atenção de startups que se apressaram para desenvolver novos aplicativos para os amantes de livros.

Você recebeu um banco de dados de um dos serviços concorrentes nesse mercado. Ele contém dados sobre livros, editoras, autores, e classificação de clientes e avaliação de livros. Essa informação será usada para gerar uma proposição válida para o novo produto.

### Descrição dos dados

**`books` — livros:**

Contém dados sobre livros:

- `book_id` — identificador do livro
- `author_id` — identificador do autor
- `title` — título
- `num_pages` — número de páginas
- `publication_date` — data de publicação
- `publisher_id` — identificador da editora

**`authors` — autores:**

Contém dados sobre os autores:

- `author_id` — identificador do autor
- `author` — autor

**`publishers` — editoras:**

Contém dados sobre editoras:

- `publisher_id` — identificador da editora
- `publisher` — editora

**`ratings` — classificações:**

Contém dados sobre classificação dos usuários:

- `rating_id` — identificador da classificação
- `book_id` — identificador do livro
- `username` — o nome do usuário que avaliou o livro
- `rating` — classificação

**`reviews` — avaliação:**

Contém dados sobre revisão dos clientes:

- `review_id` — identificador da revisão
- `book_id` — identificador do livro
- `username` — o nome do usuário que revisou o livro
- `text` — o texto da revisão

### Tarefas

- Encontre o número de livros lançados depois de 1 de janeiro de 2000.
- Encontre o número de avaliações e a classificação média para cada livro.
- Identifique a editora que lançou o maior número de livros com mais de 50 páginas (isso vai ajudar você a excluir brochuras e publicações parecidas da sua análise).
- Identifique o autor com a média mais alta classificação de livros: olhe apenas para livros com pelo menos 50 classificações.
- Encontre o número médio de avaliações entre usuários que classificaram mais do que 50 livros.

#### 1. Importando bibliotecas

In [2]:
import pandas as pd
from sqlalchemy import create_engine 

#### 2. Conectando-se ao banco de dados 

In [3]:
db_config = {'user': 'practicum_student', # username
             'pwd': 'QnmDH8Sc2TQLvy2G3Vvh7', # password
             'host': 'yp-trainers-practicum.cluster-czs0gxyx2d8w.us-east-1.rds.amazonaws.com',
             'port': 5432, # connection port
             'db': 'data-analyst-final-project-db' # the name of the database
             }
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config ['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

### 3. Queries

##### Estudando as tabelas 

In [4]:
query = ''' SELECT * 
            FROM books
            LIMIT 5'''

books = pd.io.sql.read_sql(query, con = engine)

print(books)

   book_id  author_id                                              title  \
0        1        546                                       'Salem's Lot   
1        2        465                 1 000 Places to See Before You Die   
2        3        407  13 Little Blue Envelopes (Little Blue Envelope...   
3        4         82  1491: New Revelations of the Americas Before C...   
4        5        125                                               1776   

   num_pages publication_date  publisher_id  
0        594       2005-11-01            93  
1        992       2003-05-22           336  
2        322       2010-12-21           135  
3        541       2006-10-10           309  
4        386       2006-07-04           268  


In [5]:
query = ''' SELECT * 
            FROM authors
            LIMIT 5'''

authors = pd.io.sql.read_sql(query, con = engine)

print(authors)

   author_id                          author
0          1                      A.S. Byatt
1          2  Aesop/Laura Harris/Laura Gibbs
2          3                 Agatha Christie
3          4                   Alan Brennert
4          5        Alan Moore/David   Lloyd


In [6]:
query = ''' SELECT * 
            FROM publishers
            LIMIT 5'''

publishers = pd.io.sql.read_sql(query, con = engine)

print(publishers)

   publisher_id                          publisher
0             1                                Ace
1             2                           Ace Book
2             3                          Ace Books
3             4                      Ace Hardcover
4             5  Addison Wesley Publishing Company


In [7]:
query = ''' SELECT * 
            FROM ratings
            LIMIT 5'''

ratings = pd.io.sql.read_sql(query, con = engine)

print(ratings)

   rating_id  book_id       username  rating
0          1        1     ryanfranco       4
1          2        1  grantpatricia       2
2          3        1   brandtandrea       5
3          4        2       lorichen       3
4          5        2    mariokeller       2


In [8]:
query = ''' SELECT * 
            FROM reviews
            LIMIT 5'''

reviews = pd.io.sql.read_sql(query, con = engine)

print(reviews)

   review_id  book_id       username  \
0          1        1   brandtandrea   
1          2        1     ryanfranco   
2          3        2       lorichen   
3          4        3  johnsonamanda   
4          5        3    scotttamara   

                                                text  
0  Mention society tell send professor analysis. ...  
1  Foot glass pretty audience hit themselves. Amo...  
2  Listen treat keep worry. Miss husband tax but ...  
3  Finally month interesting blue could nature cu...  
4  Nation purpose heavy give wait song will. List...  


##### Encontre o número de livros lançados depois de 1 de janeiro de 2000.

In [28]:
query = ''' SELECT COUNT(book_id) AS book_id_cnt, COUNT(DISTINCT book_id) AS book_id_unique_cnt
            FROM books 
            WHERE publication_date > '2000-01-01'
        '''

task_1 = pd.io.sql.read_sql(query, con = engine)

print(task_1)

   book_id_cnt  book_id_unique_cnt
0          819                 819


Temos 819 livros lançados após 1 de janeiro de 2000

##### Encontre o número de avaliações e a classificação média para cada livro

In [27]:
query = ''' SELECT b.book_id, b.title, COUNT(r.rating) AS rating_cnt, AVG(r.rating) AS rating_avg
            FROM books AS b
            INNER JOIN ratings AS r ON r.book_id = b.book_id
            GROUP BY b.book_id
            ORDER BY rating_cnt DESC;
        '''
        
task_2 = pd.io.sql.read_sql(query, con = engine)

print(task_2)

     book_id                                              title  rating_cnt  \
0        948                            Twilight (Twilight  #1)         160   
1        750                The Hobbit  or There and Back Again          88   
2        673                             The Catcher in the Rye          86   
3         75               Angels & Demons (Robert Langdon  #1)          84   
4        302  Harry Potter and the Prisoner of Azkaban (Harr...          82   
..       ...                                                ...         ...   
995      465                  Naked Empire (Sword of Truth  #8)           2   
996       55          A Woman of Substance (Emma Harte Saga #1)           2   
997      652          The Body in the Library (Miss Marple  #3)           2   
998      790  The Magicians' Guild (Black Magician Trilogy  #1)           2   
999      191  Disney's Beauty and the Beast (A Little Golden...           1   

     rating_avg  
0      3.662500  
1      4.125000

##### Identifique a editora que lançou o maior número de livros com mais de 50 páginas (isso vai ajudar você a excluir brochuras e publicações parecidas da sua análise).

In [26]:
query = ''' SELECT p.publisher_id , p.publisher, COUNT(b.book_id) AS books_cnt
            FROM publishers AS p
            INNER JOIN books AS b ON b.publisher_id = p.publisher_id
            WHERE b.num_pages > 50
            GROUP BY p.publisher_id
            ORDER BY books_cnt DESC
            LIMIT 5
        '''
        
task_3 = pd.io.sql.read_sql(query, con = engine)

print(task_3)

   publisher_id                 publisher  books_cnt
0           212             Penguin Books         42
1           309                   Vintage         31
2           116  Grand Central Publishing         25
3           217          Penguin Classics         24
4            35                    Bantam         19


A editora que lançou o maior número de livros com mais de 50 paginas foi a editora Penguin Books com 42 livros 

##### Identifique o autor com a média mais alta classificação de livros: olhe apenas para livros com pelo menos 50 classificações.

In [23]:
#firs: first find rating avg and rating count per book to filter books with at least 50 ratings 
#second: merging tables: books with authors and books with ratings filtered, table info still per book 
#third: average rating of all books per author
query = ''' 
WITH rating_ AS (
    SELECT book_id, COUNT(rating) AS rating_cnt, AVG(rating) AS rating_avg
    FROM ratings
    GROUP BY book_id
    HAVING COUNT(rating) > 50
    ), books_authors_ratings AS (
        SELECT b.book_id, a.author_id, a.author, rating_cnt, rating_avg
        FROM books AS b
        INNER JOIN authors AS a ON a.author_id = b.author_id
        INNER JOIN rating_ AS r ON r.book_id = b.book_id)

SELECT author_id, author, AVG(rating_avg) AS author_rating
FROM books_authors_ratings
GROUP BY author_id, author
ORDER BY author_rating DESC
LIMIT 5
        '''
        
task_4 = pd.io.sql.read_sql(query, con = engine)

print(task_4)

   author_id                             author  author_rating
0        236         J.K. Rowling/Mary GrandPré       4.283844
1        402  Markus Zusak/Cao Xuân Việt Khương       4.264151
2        240                     J.R.R. Tolkien       4.258446
3        376                  Louisa May Alcott       4.192308
4        498                       Rick Riordan       4.080645


J.K. Rowling/Mary GrandPré são os autores melhores avaliados 

##### Encontre o número médio de avaliações entre usuários que classificaram mais do que 50 livros.

In [ ]:
#first filter user with more than 50 ratings
#second count reviews of each user and inner join with filtered rating table through username to filter only user with 50+ ratings 
#third AVG of the final table 

query = ''' 
        WITH users_50_ratings AS (
        SELECT username, COUNT(rating_id) AS rating_cnt 
        FROM ratings 
        GROUP BY username
        HAVING COUNT(rating_id) > 50
        ), count_reviews AS(
                SELECT r.username, COUNT(r.review_id) AS review_cnt 
                FROM reviews AS r
                INNER JOIN users_50_ratings AS u ON u.username = r.username
                GROUP BY r.username
        )

SELECT AVG(review_cnt) AS avg_review 
FROM count_reviews;
        '''
        
task_5 = pd.io.sql.read_sql(query, con = engine)

print(task_5)

   avg_review
0   24.333333


Dos usuários que classificaram mais do que 50 livros, o número médio de avaliações é 24